In [35]:
from huggingface_hub import InferenceClient
from tqdm import tqdm
import os
import json

In [4]:
from concurrent.futures import ThreadPoolExecutor, as_completed

generate_kwargs = dict(
    temperature=1.0,
    max_new_tokens=2048,
    top_p=0.95,
    repetition_penalty=1.0,
    do_sample=True,
)

In [5]:
client = InferenceClient(
    ""
)


def format_prompt(message, history):
  prompt = "<s>"
  for user_prompt, bot_response in history:
    prompt += f"[INST] {user_prompt} [/INST]"
    prompt += f" {bot_response}</s> "
  prompt += f"[INST] {message} [/INST]"
  return prompt

In [53]:
topics = [
    'hubungan malaysia dengan china',
    'hubungan malaysia dengan US',
    'hubungan malaysia dengan palestin',
    'hubungan malaysia dengan israel',
    'hubungan malaysia dengan russia',
    'hubungan malaysia dengan singapore',
    'pendapatan B40',
    'pendapatan M40',
    'pendapatan T20',
    'miskin tegar',
    'tanaman padi',
    'pengeluaran beras',
    'eksport getah',
    'minyak sawit',
    'durian',
    'manggis',
    'kelapa',
    'tembakau',
    'rambutan',
    'nenas',
    'pisang emas',
    'dr mahathir mohamad',
    'anwar ibrahim',
    'najib razak',
    'ismail sabri yaakob',
    'muhyiddin yasin',
    'rafizi ramli',
    'Khairy Jamaluddin',
    'nasi lemak',
    'nasi goreng',
    'teh tarik',
    'teh ais',
    'kopi ais',
    'laksa',
    'kuih muih',
    'sate',
    'roti canai',
]

prompts = []
for t in topics:
    prompt = f'generate complex malay questions related to {t} in malaysian context'
    prompts.extend([prompt] * 50)
    
len(prompts)

1850

In [54]:
!mkdir mixtral-hubungan

mkdir: cannot create directory ‘mixtral-hubungan’: File exists


In [55]:
def answer(q, i):
    filename = f'mixtral-hubungan/{i}.json'
    if os.path.exists(filename):
        return
    
    while True:
        try:
            prompt = q
            formatted_prompt = format_prompt(prompt, [])
            stream = client.text_generation(formatted_prompt, **generate_kwargs, stream=False, details=True, return_full_text=False)
            output = stream.generated_text
            with open(filename, 'w') as fopen:
                json.dump(output, fopen)
            break
        except:
            pass

In [ ]:
max_worker = 50

questions = prompts
for i in tqdm(range(0, len(questions), max_worker)):
    urls_ = [(q, no + i) for no, q in enumerate(questions[i: i + max_worker])]
    
    with ThreadPoolExecutor(max_workers=max_worker) as executor:
        futures = {executor.submit(answer, url[0], url[1]): url for url in urls_}

        for future in as_completed(futures):
            future.result()

 57%|███████████████████████▊                  | 21/37 [00:15<00:00, 208.77it/s]